<a href="https://colab.research.google.com/github/v3dg/Arbitrage2/blob/main/Arb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title Code
import itertools
import requests
import pandas as pd
from datetime import datetime

# Set your API key directly in the code
API_KEY = 'b161f22b739552747ca4b6a72b90da3a'

# Bookmaker regions
REGIONS = 'us,us2'

# Odds markets
MARKETS = 'h2h'

# Odds format
ODDS_FORMAT = 'decimal'

# Date format
DATE_FORMAT = 'iso'

# List of sports
sports = [
    'americanfootball_cfl',
    'americanfootball_ncaaf',
    'americanfootball_nfl',
    'americanfootball_ufl',
    'aussierules_afl',
    'baseball_mlb',
    'baseball_mlb_preseason',
    'baseball_npb',
    'baseball_kbo',
    'baseball_ncaa',
    'basketball_euroleague',
    'basketball_nba',
    'basketball_wnba',
    'basketball_ncaab',
    'boxing_boxing',
    'cricket_big_bash',
    'cricket_caribbean_premier_league',
    'cricket_icc_world_cup',
    'cricket_international_t20',
    'cricket_ipl',
    'cricket_odi',
    'cricket_psl',
    'cricket_t20_blast',
    'cricket_test_match',
    'icehockey_nhl',
    'icehockey_sweden_allsvenskan',
    'icehockey_sweden_hockey_league',
    'mma_mixed_martial_arts',
    'rugbyleague_nrl',
    'soccer_africa_cup_of_nations',
    'soccer_argentina_primera_division',
    'soccer_australia_aleague',
    'soccer_austria_bundesliga',
    'soccer_belgium_first_div',
    'soccer_brazil_campeonato',
    'soccer_brazil_serie_b',
    'soccer_chile_campeonato',
    'soccer_china_superleague',
    'soccer_denmark_superliga',
    'soccer_england_efl_cup',
    'soccer_england_league1',
    'soccer_england_league2',
    'soccer_epl',
    'soccer_fa_cup',
    'soccer_fifa_world_cup',
    'soccer_fifa_world_cup_womens',
    'soccer_finland_veikkausliiga',
    'soccer_france_ligue_one',
    'soccer_france_ligue_two',
    'soccer_germany_bundesliga',
    'soccer_germany_bundesliga2',
    'soccer_germany_liga3',
    'soccer_greece_super_league',
    'soccer_italy_serie_a',
    'soccer_italy_serie_b',
    'soccer_japan_j_league',
    'soccer_korea_kleague1',
    'soccer_league_of_ireland',
    'soccer_mexico_ligamx',
    'soccer_netherlands_eredivisie',
    'soccer_norway_eliteserien',
    'soccer_poland_ekstraklasa',
    'soccer_portugal_primeira_liga',
    'soccer_spain_la_liga',
    'soccer_spain_segunda_division',
    'soccer_spl',
    'soccer_sweden_allsvenskan',
    'soccer_sweden_superettan',
    'soccer_switzerland_superleague',
    'soccer_turkey_super_league',
    'soccer_uefa_europa_conference_league',
    'soccer_uefa_champs_league',
    'soccer_uefa_euro_qualification',
    'soccer_uefa_europa_conference_league',
    'soccer_uefa_europa_league',
    'soccer_uefa_nations_league',
    'soccer_conmebol_copa_america',
    'soccer_conmebol_copa_libertadores',
    'soccer_usa_mls',
    'tennis_atp_aus_open_singles',
    'tennis_atp_french_open',
    'tennis_atp_us_open',
    'tennis_atp_wimbledon',
    'tennis_wta_aus_open_singles',
    'tennis_wta_french_open',
    'tennis_wta_us_open',
    'tennis_wta_wimbledon'
]

# Function to calculate arbitrage
def calculate_arbitrage(odds):
    total_inverse_prob = 0
    for outcome in odds:
        total_inverse_prob += 1 / outcome
    return total_inverse_prob

# Function to check if the day and month match
def is_same_day_and_month(date1, date2):
    return date1.month == date2.month and date1.day == date2.day

# Prepare lists to collect data
data = []

# Loop through each sport and fetch odds
for SPORT in sports:
    # Now get a list of live & upcoming games for the current sport, along with odds for different bookmakers
    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds', params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    })

    if odds_response.status_code != 200:
        print(f'Failed to get odds for {SPORT}: status_code {odds_response.status_code}, response body {odds_response.text}')
    else:
        odds_json = odds_response.json()
        print(f'Number of events for {SPORT}: {len(odds_json)}')

        # Print the remaining requests
        print('Remaining requests:', odds_response.headers['x-requests-remaining'])

        for event in odds_json:
            outcomes = {}

            for bookmaker in event['bookmakers']:
                outcomes[bookmaker['title']] = {
                    'home': bookmaker['markets'][0]['outcomes'][0]['price'],
                    'away': bookmaker['markets'][0]['outcomes'][1]['price']
                }

                if len(bookmaker['markets'][0]['outcomes']) > 2:
                    outcomes[bookmaker['title']]['tie'] = bookmaker['markets'][0]['outcomes'][2]['price']

            for bookmaker1, bookmaker2 in itertools.combinations(outcomes.keys(), 2):
                odds1 = outcomes[bookmaker1]['home']
                odds2 = outcomes[bookmaker2]['away']

                if 'tie' in outcomes[bookmaker1]:
                    odds_tie = outcomes[bookmaker1]['tie']
                    total_inverse_prob = calculate_arbitrage([odds1, odds2, odds_tie])
                else:
                    total_inverse_prob = calculate_arbitrage([odds1, odds2])

                if total_inverse_prob < 1:
                    bet_amount1 = 100 * (1 / odds1) / total_inverse_prob
                    bet_amount2 = 100 * (1 / odds2) / total_inverse_prob

                    original_minimum_profit = 100 / total_inverse_prob - 100

                    # Round bet amounts to the nearest dollar
                    bet_amount1_rounded = round(bet_amount1)
                    bet_amount2_rounded = round(bet_amount2)

                    if 'tie' in outcomes[bookmaker1]:
                        bet_amount_tie = 100 * (1 / odds_tie) / total_inverse_prob
                        bet_amount_tie_rounded = round(bet_amount_tie)
                    else:
                        bet_amount_tie_rounded = 0

                    # Recalculate the total investment and new profits
                    total_investment = bet_amount1_rounded + bet_amount2_rounded + (bet_amount_tie_rounded if 'tie' in outcomes[bookmaker1] else 0)

                    profit1 = bet_amount1_rounded * odds1 - total_investment
                    profit2 = bet_amount2_rounded * odds2 - total_investment
                    profit_tie = bet_amount_tie_rounded * odds_tie - total_investment if 'tie' in outcomes[bookmaker1] else float('-inf')


                    if profit1 > 0 and profit2 > 0 and (profit_tie > 0 if 'tie' in outcomes[bookmaker1] else True):
                            event_data = {
                                'Name of Match': f"{event['home_team']} vs. {event['away_team']}",
                                'Sportsbook 1': bookmaker1,
                                'Sportsbook 2': bookmaker2,
                                'Sportsbook 3 (if tie)': bookmaker1 if 'tie' in outcomes[bookmaker1] else None,
                                'Odds 1': odds1,
                                'Odds 2': odds2,
                                'Odds 3 (if tie)': odds_tie if 'tie' in outcomes[bookmaker1] else None,
                                'Amount on Odd 1 (Unrounded)': bet_amount1,
                                'Amount on Odd 2 (Unrounded)': bet_amount2,
                                'Amount on Odd 3 (Unrounded)': bet_amount_tie if 'tie' in outcomes[bookmaker1] else None,
                                'Minimum Profit (Unrounded)': original_minimum_profit,
                                'Amount on Odd 1': bet_amount1_rounded,
                                'Amount on Odd 2': bet_amount2_rounded,
                                'Amount on Odd 3': bet_amount_tie_rounded if 'tie' in outcomes[bookmaker1] else None,
                                'Profit if Odd 1 Wins': profit1,
                                'Profit if Odd 2 Wins': profit2,
                                'Profit if Odd 3 Wins': profit_tie if 'tie' in outcomes[bookmaker1] else None,
                                'Date added': datetime.now().isoformat(),
                                'Date of Game': event['commence_time'],
                                'Test2': 2  # Adding the Test column with all values equal to 0
                            }
                            data.append(event_data)

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('arbitrage_opportunities.csv', index=False)

print('Data saved to arbitrage_opportunities.csv')

# Read CSV and convert to HTML
df = pd.read_csv('arbitrage_opportunities.csv')
html_content = df.to_html(index=False)

# HTML structure with linked CSS
html_page = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Arbitrage Opportunities</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
<h1>Arbitrage Opportunities</h1>

<p>Welcome to ArbitrageXpert!</p>

<p>Thank you for choosing our ArbitrageXpert package. We assure you it's a decision that will prove worthwhile. Before you dive into finding lucrative arbitrage opportunities, let us reassure you: arbitrage betting is completely legal and a legitimate way to capitalize on market inefficiencies.</p>
<p>**Instructions:**</p>
<p>1. **Review the Live Data:**
   Once you've finished reading these instructions, take a look at the live data below. You'll find various arbitrage opportunities listed, each showing profit margins, associated sportsbooks, and other pertinent details.</p>
<p>2. **Choosing a Match:**
   - Identify opportunities with higher profit margins—they offer greater potential returns.
   - Consider the timing of the match. Rapidly changing odds in live or imminent games mean swift action is crucial to secure profitable bets.
   - Focus on major sports leagues. Sportsbooks may restrict or ban users exploiting arbitrage in lesser-known leagues.
   - Ensure the selected sportsbooks operate legally in your region. Some platforms may be restricted based on local regulations.</p>
<p>3. **Placing Your Bets:**
   - Opt for rounded odds when placing bets. This minimizes the risk of detection by sportsbooks, who may limit accounts engaging in arbitrage with non-rounded numbers.
   - Note that rounding may slightly skew potential winnings towards one outcome. However, rest assured that our selections are strategically chosen to ensure profitability regardless of the match's outcome. Detailed graphs below illustrate this balance.</p>
<p>
4. **Final Steps:**
   - Once bets are placed, your part is complete! Enjoy the game and await the outcome.
   - After the match, simply cash out your winnings.</p>
<p>We sincerely appreciate your trust in ArbitrageXpert. Should you have any questions or need assistance, our support team is here to help. Remember, this process becomes straightforward once you familiarize yourself with it.</p>
<p>Thank you again for choosing ArbitrageXpert. Get ready to capitalize on arbitrage opportunities and enjoy the rewards!</p>
<p>Best regards,</p>
<p>ArbitrageXpert</p>

</body>
<body>
    <h1>Arbitrage Opportunities Chart</h1>
    {html_content}
</body>
</html>
"""

# Save the HTML content to a file
with open('grades.html', 'w') as file:
    file.write(html_page)

print('HTML file created as grades.html')
print('r')






Number of events for americanfootball_cfl: 4
Remaining requests: 15712
Number of events for americanfootball_ncaaf: 123
Remaining requests: 15710
Number of events for americanfootball_nfl: 272
Remaining requests: 15708
Number of events for americanfootball_ufl: 0
Remaining requests: 15708
Number of events for aussierules_afl: 15
Remaining requests: 15706
Number of events for baseball_mlb: 20
Remaining requests: 15704
Number of events for baseball_mlb_preseason: 0
Remaining requests: 15704
Number of events for baseball_npb: 0
Remaining requests: 15704
Number of events for baseball_kbo: 5
Remaining requests: 15702
Number of events for baseball_ncaa: 2
Remaining requests: 15700
Number of events for basketball_euroleague: 0
Remaining requests: 15700
Number of events for basketball_nba: 0
Remaining requests: 15700
Number of events for basketball_wnba: 7
Remaining requests: 15698
Number of events for basketball_ncaab: 0
Remaining requests: 15698
Number of events for boxing_boxing: 51
Remaini

# New Section

In [ ]:
import pandas as pd
df = pd.read_csv('arbitrage_opportunities.csv')
df.to_html('grades.html')


In [8]:
# @title Default title text
/* Define the overall style for the body */
body {
    font-family: Arial, sans-serif;
    margin: 20px;
    background-color: #f4f4f9;
    color: #333;
}

/* Style the table to make it more visually appealing */
table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
    box-shadow: 0 2px 3px rgba(0,0,0,0.1);
}

/* Style the table headers */
th {
    background-color: #4CAF50;
    color: #fff;
    padding: 10px;
    text-align: left;
    border-bottom: 2px solid #ddd;
}

/* Style the table cells */
td {
    padding: 10px;
    border-bottom: 1px solid #ddd;
}

/* Add alternating row colors for better readability */
tr:nth-child(even) {
    background-color: #e8f5e9;
}

/* Style the table rows on hover */
tr:hover {
    background-color: #d7ffd9;
}

/* Style the first column specifically (e.g., for highlighting match names) */
td:first-child {
    font-weight: bold;
    color: #2e7d32;
}

/* Add a header style */
h1 {
    color: #2e7d32;
    text-align: center;
    margin-bottom: 20px;
}

/* Style the table borders */
table, th, td {
    border: 1px solid #ddd;
}

SyntaxError: invalid decimal literal (<ipython-input-8-927200bde142>, line 5)

git remote add origin https://github.com/v3dg/Arbitrage2.git
